<h1 style="color: blue; text-align: center;">Multi-task Cascaded Convolutional Networks (MTCNN) for Face Detection and Facial Landmark Alignment Using Parallel</h1>

---

<h2 style="color: gray; text-align: right;">Members of Team:</h2>

| MSSV | Full Name | ID Github |
| ---- | ---- | -------- |
| 1612835 | Bùi Trọng Xuyến | btxuyenhcmus |
| 1612165 | Nguyễn Đào Vinh Hải | NDVHaiHCMUS |
| 1612859 | Nguyễn Đình Hữu | NguyenDinhHuu-HCMUS |

---
## 1.  Mô tả ứng dụng

**Input:** Một tấm ảnh RGB hoặc một video live trực tiếp.

**Output:** Một tấm ảnh RGB hoặc một video live trực tiếp được bounding các object và gắn nhãn.

**Ý nghĩa thực tế của ứng dụng:**
- bài toán face detection là một bài toán mang lại rất nhiều lợi ích hiện nay, từ việc quản lý con người, phát hiện đối tượng lạ, hay như chấm công,...
- Khi chúng ta song song hoá được bài toán này thì sẽ đem lại lợi ích cao từ việc giảm chi phí tiền máy chủ, cho đến tiếp kiệm được thời gian.

**ứng dụng này cần được tăng tốc**
- Thời gian training của các mô hình neural hầu như đều rất chậm.
- Khi xử lý với video khi lượng frame đưa xuống quá nhanh, kết quả trả về của mô hình không kiệp đáp ứng tạo nên độ trễ.
- Bài toán sử dụng convolution là chủ yếu.
- Xử lý nhiều trên matrix


![index](./images/examples.png)
> Nguồn ảnh: [https://kpzhang93.github.io/MTCNN_face_detection_alignment/](https://kpzhang93.github.io/MTCNN_face_detection_alignment/)

---
## 2. Cài đặt tuần tự
### Ý tưởng
![index](./images/index.png)

### The Three Stages of MTCNN:
Bước đầu tiên là lấy hình ảnh và thay đổi kích thước theo các tỷ lệ khác nhau để xây dựng một kim tự tháp hình ảnh, đây là đầu vào của **three-staged cascaded network**.

> **?? Tại sao chúng ta lại tạo ra một image pyramid**
> Bằng cách tăng tính đa dạng của dữ liệu, độ chính xác của mô hình MTCNN được nâng cao.

![inputimage](./images/inputimage.png)
> Input image is resized to different scales to build an image pyramid

#### Stage 1: The Proposal Network (P-Net)

![P-net](./images/P-net.jpeg)
> P-Net (from MTCNN paper)

P-net là một Fully convolutional network (FCN). Sự khác biết giữa FCN và CNN đó là FCN không sử dụng dense layer. P-net được sử dụng để lấy các cửa sổ ứng viên và các vector hồi quy bao quanh.

Bounding box regression là công nghệ phổ biến để dự đoán khu vực của một đối tượng phân lớp, trong trường hợp này là khuôn mặt. Sau khi có tất cả các bounding box thì các bounding box này có thể bị chồng chéo lẫn nhau, cần thông qua một lớp lọc để cho ra khác khu vực không bị trùng lắp.

**Input:** image RGB

**Output:** danh sách n-box vector (1x9)
***
#### Stage 2: The Refine Network (R-Net)

![R-net](./images/R-net.jpeg)
> R-Net (from MTCNN paper)

Tất cả các ứng viên từ P-net sẽ được đưa vào R-net. R-net là một CNN bởi vì có một dense layer ở cuối stage. R-net giảm thêm số lượng ứng viên, thực hiện calibration các bounding box regression và NMS (non-maximum suppression) để hợp nhất các ứng viên trùng lắp.

**Input:** output cura P-net

**Output:** Có phải là khuôn mặt hay không, một vector có 4 phần tử là bounding box của khuôn mặt, một vector 10 phần tử là các vị trí trong khuôn mặt.
***
#### Stage 3: The Output Network (O-Net)

![O-net](./images/O-net.jpeg)
> R-Net (from MTCNN paper)

Giai đoạn này tương tự như R-net, nhưng Output của network này mô tả chi tiết hơn về 5 điềm của khuông mặt mắt, mũi, miệng.

**Input:** output của R-net

**Output:** Vector có 4 phần tử là bounding box của khuôn mặt, một vector 5 phần tử các vị trí của khuôn mặt
***
### The Three Tasks of MTCNN:
Trong tất cả các stage của Networks đều thực hiện 3 task: face/non-face classification, bounding box regression, facial landmark localization.

#### 1. Face classification:
Sử dụng cross-entropy loss vì đây là bài toán phân 2 lớp, với mỗi mẫu $x_i$

$$L_i^{det} = -(y_i^{det}\log(p_i) + (1 - y_i^{det})(1-\log(p_i)))$$

Trong đó, $p_i$ là xác xuất được tạo ra bớt net chỉ ra một sample là face. $y_i^{det} \in {0, 1}$ là giá trị thật khuôn mặt hoặc không.

#### 2. Bounding box regression:
Với mỗi cửa sổ ứng viên, chúng ta dự đoán các vị trí tin tưởng tạo nên một bộ offset (left top, height, width). Việc huấn luyện là bài toán regression cho nên sử dụng Euclidean loss cho mỗi sameple $x_i$:

$$L_i^{box} = \parallel\widehat{y}_i^{box} - y_i^{box}\parallel_2^2$$

Trong đó, $\widehat{y}_i^{box}$ là các vị trí dự đoán được của net, $y_i^{box}$ là tọa độ thật. Chúng ta sẽ có bốn góc, bao gồm left top, height, width và $y_i^{box} \in R^4$

#### 3. Facial Landmark localization:
Tương tự như bài toán regression và chúng ta sẽ đi minimize Euclidean distance:

$$L_i^{landmark} = \parallel\widehat{y}_i^{landmark} - y_i^{landmark}\parallel_2^2$$

---
### 3. Cài đặt song song (trên GPU)

---
### 4. Cài đặt song song (trên GPU) + tối ưu hóa

---
### 5. Nhìn lại quá trình làm đồ án

---
### 6. Tài liệu tham khảo

- [x] [PDF report](https://arxiv.org/pdf/1604.02878.pdf)
- [x] [Document Guide](https://medium.com/@iselagradilla94/multi-task-cascaded-convolutional-networks-mtcnn-for-face-detection-and-facial-landmark-alignment-7c21e8007923)
- [x] [MTCNN-Pytorch](https://github.com/TropComplique/mtcnn-pytorch)